In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import os
import cv2
import GPUtil
import matplotlib.pyplot as plt                        
%matplotlib inline  

# In a multi GPU server, chose which to use:
NUMBER_OF_GPUS_TO_USE = 1
Availability=GPUtil.getAvailability(GPUtil.getGPUs())
all_gpus = np.arange(3)
available_gpu_indexes = [x for x in all_gpus if Availability[x]]
# Set CUDA_DEVICE_ORDER so the IDs assigned by CUDA match those from nvidia-smi
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# Set CUDA_VISIBLE_DEVICES to mask out all other GPUs than the first NUMBER_OF_GPUS_TO_USE available device id
os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(np.array(available_gpu_indexes[:NUMBER_OF_GPUS_TO_USE]).astype(str))

import tensorflow as tf
config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

import keras
from keras.backend.tensorflow_backend import set_session

set_session(tf.Session(config=config))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image                  
from tqdm import tqdm

from keras.models import Sequential
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.core import Dense

from keras.applications.resnet50 import preprocess_input, decode_predictions

from sklearn.datasets import load_files       
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from glob import glob

from keras.engine.input_layer import Input
from keras.models import Model

from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import ModelCheckpoint

In [4]:
import itertools

In [5]:
import string, random
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

### Confirm the image is actually a dog

The ability of this process is limited by the dataset

In [6]:
def path_to_tensor(img_path, expand=True):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    if expand:
        return np.expand_dims(x, axis=0)
    else:
        return x

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [5]:
# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

In [7]:
def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151)) 

### Use transfer learning to reduce training time

Model architecture

In [7]:
### Define architecture.
Resnet50_model = Sequential()
Resnet50_model.add(GlobalAveragePooling2D(input_shape=(7, 7, 2048)))
Resnet50_model.add(Dense(120, activation='softmax'))

In [8]:
Resnet50_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               245880    
Total params: 245,880
Trainable params: 245,880
Non-trainable params: 0
_________________________________________________________________


#### Compile and test

##### Load data

In [9]:
data = load_files("/notebooks/Practice/imgnet-dogbreed/Images/")

In [10]:
train_files, test_files, train_targets, test_targets = train_test_split(data['filenames'], data['target'], test_size=0.1)
train_targets = np_utils.to_categorical(np.array(train_targets), 120)
test_targets = np_utils.to_categorical(np.array(test_targets), 120)

dog_names = [item.split('/')[-1] for item in sorted(glob("/notebooks/Practice/imgnet-dogbreed/Images/*"))]

In [11]:
# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d test dog images.'% len(test_files))

There are 120 total dog categories.
There are 20580 total dog images.

There are 18522 training dog images.
There are 2058 test dog images.


Create bottleneck features

In [26]:
def random_crop(image):
    """Crops a random region of width x height from image.
    Returns an image"""
    ratio = 0.8
    width = int(image.shape[1] * ratio)
    height = int(image.shape[0] * ratio)
    crop_origin_x = np.random.randint(0, image.shape[0] - width)
    crop_origin_y = np.random.randint(0, image.shape[1] - height)
    cropped_image = image[crop_origin_y:crop_origin_y+height,crop_origin_x:crop_origin_x+width,:]
    cropped_image = cv2.resize(cropped_image,(image.shape[0],image.shape[1]))
    return cropped_image

In [32]:
# create and configure augmented image generator
datagen_train = ImageDataGenerator(preprocessing_function=random_crop,
                                   rotation_range=45,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

# create and configure augmented image generator
datagen_test = ImageDataGenerator() 

In [13]:
processed_train_files = preprocess_input(paths_to_tensor(train_files))
processed_test_files = preprocess_input(paths_to_tensor(test_files))

In [10]:
train_targets = np.load('ResNet50/train_targets.npy')
test_targets = np.load('ResNet50/test_targets.npy')

train_files = np.load('ResNet50/train_files.npy')
test_files = np.load('ResNet50/test_files.npy')

processed_train_files = np.load('ResNet50/processed_train_files.npy')
processed_test_files = np.load('ResNet50/processed_test_files.npy')

In [33]:
# fit augmented image generator on data
datagen_train.fit(processed_train_files)
datagen_test.fit(processed_test_files)

In [15]:
input_img = Input(shape=(224, 224, 3))

model_resnet = ResNet50(weights='imagenet', include_top=False,
    input_shape=(224, 224, 3),
    input_tensor=input_img)

In [34]:
train_flow = datagen_train.flow(processed_train_files, train_targets, shuffle=True, batch_size=3000)
test_flow = datagen_test.flow(processed_test_files, test_targets, shuffle=False, batch_size=len(test_files))

In [35]:
Resnet50_model.compile(optimizer=keras.optimizers.Nadam(),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=[keras.metrics.categorical_accuracy])
### Train the model.
checkpointer_resnet = ModelCheckpoint(filepath='ResNet50/weights.best.ResNet50.hdf5', 
                               verbose=0, save_best_only=True)

tb = keras.callbacks.TensorBoard(log_dir='./logs'+'/'+id_generator(), histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)

In [ ]:
for x,y in train_flow:
    history = Resnet50_model.fit(model_resnet.predict(x), y, epochs=5, callbacks=[checkpointer_resnet, tb], validation_split=.1, verbose=0)
    if history.history['val_categorical_accuracy'][0] > .9:
        break
    

In [ ]:
%%time 

test_features = model_resnet.predict(processed_test_files)

### Calculate classification accuracy on the test dataset.
predictions = np.argmax(Resnet50_model.predict(test_features), axis=1)
actual = np.argmax(test_targets, axis=1)
# Report test accuracy
test_accuracy = 100*np.sum(predictions==actual)/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

In [41]:
np.save('ResNet50/train_targets', train_targets)
np.save('ResNet50/test_targets', test_targets)

np.save('ResNet50/train_files',train_files)
np.save('ResNet50/test_files',test_files)

np.save('ResNet50/processed_train_files', processed_train_files)
np.save('ResNet50/processed_test_files', processed_test_files)

In [29]:
for x,y in validation_flow:
    print(np.shape(x),np.shape(y))

(32, 224, 224, 3) (32, 120)
(32, 224, 224, 3) (32, 120)
(32, 224, 224, 3) (32, 120)
(32, 224, 224, 3) (32, 120)
(32, 224, 224, 3) (32, 120)
(32, 224, 224, 3) (32, 120)
(32, 224, 224, 3) (32, 120)
(32, 224, 224, 3) (32, 120)
(32, 224, 224, 3) (32, 120)
(32, 224, 224, 3) (32, 120)
(32, 224, 224, 3) (32, 120)
(32, 224, 224, 3) (32, 120)


KeyboardInterrupt: 

In [31]:
%time train_features = model_resnet.predict(x)

CPU times: user 117 ms, sys: 60.5 ms, total: 177 ms
Wall time: 155 ms


In [27]:
%time train_features = model_resnet.predict_generator(train_flow, steps=1)

CPU times: user 527 ms, sys: 64.3 ms, total: 591 ms
Wall time: 459 ms


In [35]:
%time test_features = model_resnet.predict_generator(test_flow, steps=1)

CPU times: user 12.5 s, sys: 573 ms, total: 13.1 s
Wall time: 12.8 s


In [36]:
%time validation_features = model_resnet.predict_generator(validation_flow, steps=1)

(64, 7, 7, 2048)

In [16]:
input_img = Input(shape=(224, 224, 3))

model_resnet = ResNet50(weights='imagenet', include_top=False,
    input_shape=(224, 224, 3),
    input_tensor=input_img)

In [164]:
%time train_features = model_resnet.predict(preprocess_input(paths_to_tensor(train_files)))

CPU times: user 2min 47s, sys: 28.8 s, total: 3min 15s
Wall time: 3min 9s


In [165]:
%time test_features = model_resnet.predict(preprocess_input(paths_to_tensor(test_files)))

CPU times: user 20.4 s, sys: 3.01 s, total: 23.4 s
Wall time: 22.6 s


In [166]:
%time validation_features = model_resnet.predict(preprocess_input(paths_to_tensor(validation_files)))

CPU times: user 20.4 s, sys: 2.94 s, total: 23.3 s
Wall time: 22.5 s


In [167]:
[[np.shape(train_features), np.shape(test_features), np.shape(validation_features)],
 [np.shape(train_targets), np.shape(test_targets), np.shape(validation_targets)]]

[[(16464, 7, 7, 2048), (2058, 7, 7, 2048), (2058, 7, 7, 2048)],
 [(16464, 120), (2058, 120), (2058, 120)]]

In [34]:
np.save('ResNet50/train_features', train_features)
np.save('ResNet50/test_features', test_features)
np.save('ResNet50/validation_features', validation_features)

np.save('ResNet50/train_targets', train_targets)
np.save('ResNet50/test_targets', test_targets)
np.save('ResNet50/validation_targets', validation_targets)

np.save('ResNet50/train_files',train_files)
np.save('ResNet50/test_files',test_files)
np.save('ResNet50/validation_files',validation_files)

np.save('ResNet50/processed_train_files', processed_train_files)
np.save('ResNet50/processed_test_files', processed_test_files)
np.save('ResNet50/processed_validation_files.npy', processed_validation_files)

/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image.py:1183: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (16464, 7, 7, 2048) (2048 channels).
  ' channels).')
/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image.py:1183: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (2058, 7, 7, 2048) (2048 channels).
  ' channels).')


In [21]:
batch_size = 50
epochs = 50

Resnet50_model.fit_generator(datagen_train.flow(train_features, train_targets, batch_size=batch_size),
                    steps_per_epoch=train_features.shape[0] // batch_size,
                    epochs=epochs, callbacks=[checkpointer_resnet, tb],
                    validation_data=datagen_valid.flow(validation_features, validation_targets, batch_size=batch_size),
                    validation_steps= validation_features.shape[0] // batch_size)

/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image.py:1404: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (16464, 7, 7, 2048) (2048 channels).
  str(self.x.shape[channels_axis]) + ' channels).')
/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image.py:1404: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (2058, 7, 7, 2048) (2048 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


Epoch 1/50
329/329 [==============================] - 764s 2s/step - loss: 1.4092 - categorical_accuracy: 0.6268 - val_loss: 0.9695 - val_categorical_accuracy: 0.7298

Epoch 00001: val_loss improved from inf to 0.96954, saving model to ResNet50/weights.best.ResNet50.hdf5
Epoch 2/50
329/329 [==============================] - 770s 2s/step - loss: 0.5373 - categorical_accuracy: 0.8359 - val_loss: 0.9941 - val_categorical_accuracy: 0.7132

Epoch 00002: val_loss did not improve from 0.96954
Epoch 3/50
329/329 [==============================] - 774s 2s/step - loss: 0.3113 - categorical_accuracy: 0.9038 - val_loss: 0.9828 - val_categorical_accuracy: 0.7410

Epoch 00003: val_loss did not improve from 0.96954
Epoch 4/50
329/329 [==============================] - 779s 2s/step - loss: 0.1960 - categorical_accuracy: 0.9392 - val_loss: 0.9826 - val_categorical_accuracy: 0.7546

Epoch 00004: val_loss did not improve from 0.96954
Epoch 5/50
329/329 [==============================] - 786s 2s/step - lo

In [26]:
### Compile the model.
Resnet50_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [19]:
Resnet50_model.compile(optimizer=keras.optimizers.Nadam(),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=[keras.metrics.categorical_accuracy])

In [20]:
### Train the model.
checkpointer_resnet = ModelCheckpoint(filepath='ResNet50/weights.best.ResNet50.hdf5', 
                               verbose=1, save_best_only=True)

tb = keras.callbacks.TensorBoard(log_dir='./logs'+'/'+id_generator(), histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)

In [53]:
Resnet50_model.fit(train_features, train_targets, 
          validation_data=(validation_features, validation_targets),
          epochs=100, batch_size=50, callbacks=[checkpointer_resnet, tb], verbose = 0)


Epoch 00001: val_loss improved from inf to 2.97892, saving model to ResNet50/weights.best.ResNet50.hdf5

Epoch 00002: val_loss improved from 2.97892 to 2.96905, saving model to ResNet50/weights.best.ResNet50.hdf5

Epoch 00003: val_loss improved from 2.96905 to 2.91374, saving model to ResNet50/weights.best.ResNet50.hdf5

Epoch 00004: val_loss improved from 2.91374 to 2.89610, saving model to ResNet50/weights.best.ResNet50.hdf5

Epoch 00005: val_loss did not improve from 2.89610

Epoch 00006: val_loss did not improve from 2.89610

Epoch 00007: val_loss improved from 2.89610 to 2.79224, saving model to ResNet50/weights.best.ResNet50.hdf5

Epoch 00008: val_loss did not improve from 2.79224

Epoch 00009: val_loss did not improve from 2.79224

Epoch 00010: val_loss did not improve from 2.79224

Epoch 00011: val_loss did not improve from 2.79224

Epoch 00012: val_loss did not improve from 2.79224

Epoch 00013: val_loss did not improve from 2.79224

Epoch 00014: val_loss did not improve from

KeyboardInterrupt: 

In [9]:
### Load the model weights with the best validation loss.
Resnet50_model.load_weights('ResNet50/weights.best.ResNet50.hdf5')

In [22]:
### Calculate classification accuracy on the test dataset.
predictions = np.argmax(Resnet50_model.predict(test_features), axis=1)
actual = np.argmax(test_targets, axis=1)
# Report test accuracy
test_accuracy = 100*np.sum(predictions==actual)/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 74.7328%


Test accuracy: 75.6074%


In [66]:
Resnet50_model.compile(optimizer=keras.optimizers.Nadam(),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=[keras.metrics.categorical_accuracy])
### Train the model.
checkpointer_resnet = ModelCheckpoint(filepath='ResNet50/weights.best.ResNet50.hdf5', 
                               verbose=1, save_best_only=True)

tb = keras.callbacks.TensorBoard(log_dir='./logs'+'/'+id_generator(), histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)

for x,y in train_flow:
    features = model_resnet.predict(x)
    history = Resnet50_model.fit(features, y, callbacks=[checkpointer_resnet, tb], validation_split=float(.1*9/10))

In [68]:
for x,y in train_flow:
    features = model_resnet.predict(x)
    history = Resnet50_model.fit(features, y, callbacks=[checkpointer_resnet, tb], validation_split=float(.1*9/10))

Train on 29 samples, validate on 3 samples
Epoch 1/1
29/29 [==============================] - 0s 523us/step - loss: 1.9773 - categorical_accuracy: 0.6552 - val_loss: 0.6282 - val_categorical_accuracy: 0.6667

Epoch 00001: val_loss did not improve from 0.05890
Train on 29 samples, validate on 3 samples
Epoch 1/1
29/29 [==============================] - 0s 318us/step - loss: 1.0658 - categorical_accuracy: 0.7586 - val_loss: 0.0024 - val_categorical_accuracy: 1.0000

Epoch 00001: val_loss improved from 0.05890 to 0.00240, saving model to ResNet50/weights.best.ResNet50.hdf5
Train on 29 samples, validate on 3 samples


KeyboardInterrupt: 

In [69]:
history.history

{'categorical_accuracy': [0.7586206793785095],
 'loss': [1.06578528881073],
 'val_categorical_accuracy': [1.0],
 'val_loss': [0.002400173107162118]}

In [62]:
for x,y in validation_flow:
    features = model_resnet.predict(x)
    Resnet50_model.fit(epochs=10, steps_per_epoch=10, validation_steps=10, validation_data=(features, y))

IndexError: list index out of range